# [3.3] Running evals with the inspect library

# 1️⃣ Introduction to Inspect

## Learning Objectives

- Understand how **solvers** and **scorers** in the inspect framework function
- Familiarise yourself with **plans**, which ultimately are just a list of solver functions, and then a scorer function.
- Understand how inspect uses **record_to_sample** to modify json datasets into evaluation datasets.
- Know the basics of how all of these classes come together to form a **Task** object.

In [9]:
import os
import json
from inspect_ai import Task, eval, task
from inspect_ai.log import read_eval_log
from inspect_ai.dataset import FieldSpec, json_dataset, Sample
from inspect_ai.solver._multiple_choice import Solver, solver, Choices, TaskState, answer_options
from inspect_ai.scorer import (model_graded_fact, match, answer, scorer)
from inspect_ai.scorer._metrics import (accuracy, stderr)
from inspect_ai.solver import ( chain_of_thought, generate, self_critique,system_message, multiple_choice, Generate )
from inspect_ai.model import ChatMessageUser, ChatMessageSystem, ChatMessageAssistant
import random
import jaxtyping 
from itertools import product

ImportError: cannot import name 'stderr' from 'inspect_ai.scorer._metrics' (c:\Users\styme\anaconda3\envs\arena\lib\site-packages\inspect_ai\scorer\_metrics\__init__.py)

## Inspect
Inspect (or inspect_ai) is a library written by the UK AISI in order to streamline model evaluations. Inspect provides the following in order to make writing evals easier:

- It automates the creation of logging files, and stores a lot of useful information for us easily.

- It automatically does exponential backing off, and provides max_connections functionality, so that we don't have to bother with this.

- It provides nice "scoring" functions (and some of its automated solver functions also come in handy, but mostly we will write our own).

- It provides a nice layout to view the results of our evals, so we don't have to look directly at model outputs (which can be very gross, as we will find out when we work with agents).


### Exercise - Write record_to_sample function

```c
Difficulty: 🔴⚪⚪⚪⚪
Importance: 🔵🔵🔵⚪⚪

You should spend up to 5-10 minutes on this exercise.
```

This function will take in a single element ("record") of your list of json objects, and returns a Sample object. For an example of what this function might look like, look at [this section of the inspect documentation](https://inspect.ai-safety-institute.org.uk/datasets.html#field-mapping).

The sample object is what the inspect_ai framework uses to deal with individual questions. The most important elements of the sample object are: 

- The `input` to the model. This is a `ChatMessage` list of dictionaries which contains the content which we will feed to the model during the eval consisting of:
    - A system message

    - A user message (this is the most necessary part of all)

- A `target` which represents the "correct" answer (or `answer_matching_behavior` in our context).

<details> 
<summary>Aside: Longer ChatMessage lists</summary>
<br>
For more complicated evals, we're able to do an arbitrary length list of ChatMessages including: 
<ul> 
<li>Multiple user messages and system messages.</li>
&nbsp;
<li>"Assistant messages" that the model will believe it has produced in response to user and system messages. However we can write these ourselves to trick the model.</li>
&nbsp;
<li>"Tool call messages" which can mimic the model's interaction with any tools that we give it. We'll learn more about tools in the agent evals section.</li>
</ul>
</details>

Since we are constructing a multiple choice evaluation, we will also need to feed it `choices`. Then it also takes in `metadata`, which can consist of any data we might want to store related to the question in our logs (for example, if there are multiple categories of question, and we want to be able to plot results by category).

In [2]:
def record_to_sample(record):
    return Sample(
        input = [],
        target = 
        choices = 
        metadata =
    )

SyntaxError: invalid syntax. Perhaps you forgot a comma? (1599308141.py, line 3)

Now we use the `json_dataset()` function to apply this function to the list of questions in our dataset.

In [ ]:
eval_dataset = json_dataset("Your/Dataset/Path/Here" record_to_sample())

## Scorers

Scorers are what the inspect library uses to evaluate model output. We use them to determine whether the model's answer is the `target` answer. From the [inspect documentation](https://inspect.ai-safety-institute.org.uk/scorers.html):


><br>
>Scorers generally take one of the following forms:
>
>1. Extracting a specific answer out of a model’s completion output using a variety of heuristics.
>
>2. Applying a text similarity algorithm to see if the model’s completion is close to what is set out in the `target`.
>
>3. Using another model to assess whether the model’s completion satisfies a description of the ideal answer in `target`.
>
>4. Using another rubric entirely (e.g. did the model produce a valid version of a file format, etc.)
>
><br>

Since we are writing a multiple choice benchmark, the scorer we will generally use is called `match()`. This determines whether the `target` appears at the end of the model's output (or the start, but by default it will look at the end). The code for this scorer function can be found [in the inspect_ai github](https://github.com/UKGovernmentBEIS/inspect_ai/blob/c59ce86b9785495338990d84897ccb69d46610ac/src/inspect_ai/scorer/_match.py). 

<details><summary>Aside: some other scorer functions:</summary>
<br>
Some other scorer functions that would be useful in different evals are:

- `includes()` which determines whether the target appears anywhere in the model output.

- `pattern()` which extracts the answer from the model output using a regular expression.

- `answer()` which is a scorer for model output when the output is preceded with "ANSWER:".

- `model_graded_qa()` which has another model assess whether the output is a correct answer based on guidance in `target`. This is especially useful for evals where the answer is more open-ended. Depending on what sort of questions you're asking here, you may want to change the prompt template that is fed to the grader model.
<br>
</details>

### Solvers

Solvers are functions which outline how the eval will be presented to the model. They can be used for:

- Providing a system prompt.
 
- Providing the evaluation question.

- Prompt engineering (e.g. chain of thought, asking the model to output in certain formats, telling the model how to act when answering).

- Self-critique (where the model criticises its prior answer to see if it arrives at a different conclusion).

- Providing fake assistant messages to the model (useful if you want to conduct an eval in the midst of an ongoing conversation with a model).

Some built in solvers which accomplish these are:

- `system_message()` which adds a system message to the list of ChatMessages.

- `prompt_template()` which modifies the user prompt by substituting the current prompt into the `{prompt}` placeholder witin the specified template. Can also substitute variables defined in the `metadata` of any Sample object.

- `chain_of_thought()` which modifies the user prompt by substituting the current prompt into the `